# Write an App with PromptFlow SDK that leverages a complex flow with multiple steps and Tools 

In this part of the workshop we will build a more sophisticated app leveraging different flow types and tools on promptflow...

## Create a Flow 
You can create LLM apps using a Python function or class as the entry point, which encapsulating your app logic. You can directly test or run these entries with pure code experience.

In [4]:
# Confirm Promptflow is installed

!pf -v


{
  "promptflow": "1.11.0",
  "promptflow-core": "1.11.0",
  "promptflow-devkit": "1.11.0",
  "promptflow-tracing": "1.11.0"
}

Executable '/opt/homebrew/opt/python@3.11/bin/python3.11'
Python (Darwin) 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]


In [7]:
# Create a new flow...

!pf flow init --flow flow01

Creating flow from scratch...
Creating hello.py...
Creating data.jsonl...
Creating .promptflow folder...
Creating __pycache__ folder...
Creating flow.dag.yaml...
Creating hello.jinja2...
Creating /Users/ozgurguler/Developer/Projects/aistudio-workshop-ozguler/code/flow01/requirements.txt...
Creating /Users/ozgurguler/Developer/Projects/aistudio-workshop-ozguler/code/flow01/.gitignore...
Done. Created standard flow folder: /Users/ozgurguler/Developer/Projects/aistudio-workshop-ozguler/code/flow01.
You can execute this command to test the flow, pf flow test --flow flow01 --input flow01/data.jsonl


When we run "flow init", it creates a folder with name matching flows name (in this case flow01 folder) with necessary files in it.

![Alt text](../media/80.png)

Structure of the flow folder:

- flow.dag.yaml: The flow definition with inputs/outputs, nodes, tools and variants for authoring purpose. \
It defines the structure and logic of the flow, specifying how different tools are connected and how data is passed between them. \
While defining the flow implicitly in app.py is possible, using a flow.yaml file provides several benefits, including better separation of concerns,\
improved readability and maintainability, enhanced reusability and modularity, and a more declarative approach to defining workflows.\
This separation allows for a cleaner, more organized,


- .promptflow/flow.tools.json:  contains the tools meta referenced in flow.dag.yaml.
- Source code files (.py, .jinja2): User managed, the code scripts referenced by **tools**.
- requirements.txt: Python package dependencies for this flow.

*(If the PromptFlow extension is installed, you can do CMD + K, V to visualise the flow as in the PromptFlow web UI like below.)*

![Alt text](../media/81.png)

We now have the flow skeleton in place, previously created with the *'flow init'* command. \
We will modify the flow so that it has a Python tool step which runs the Prompty file reflecting the recent updates to prompt management in Build,24. \
Let's now update the flow.dag.yaml so that it simply leverages a **python tool** that loads and runs the prompty.prompty. 

Below is the prompty file for to generate creative writing pieces. 
(make sure your AOAI endpoint crendtials and base url are added to the .env file in the folder where prompty file is stored.)

*creative_writing_assistant.prompty is saved under code/flow01 which you can copy from.*

```
---
name: Creative Writing Assistant
description: Generate creative writing prompts and outlines based on a theme.
model:
  api: chat
  configuration:
    type: azure_openai
    azure_deployment: gpt-35-turbo-16k-ozguler04
  parameters:
    temperature: 0.7
    max_tokens: 2000
inputs:
  theme:
    type: string
sample:
  theme: "Mystery and Adventure"
---

system:
You are an imaginative and supportive creative writing assistant. 
Your task is to help users generate intriguing writing prompts and provide a brief story outline based on the given theme.

user:
Theme: {{theme}}

assistant:
Here is an interesting writing prompt based on the theme "{{theme}}":
[Generate a complete story here, including an introduction, conflict, rising action, climax, falling action, and resolution, without explicitly labeling these sections.]
```


## Updating the flow.dag.yaml 
Although we ran the prompty file as a chat app in "first-pf-sdk-notebook.ipynb", we will build a more complex app step by step in this notebook. \
Our app will receive a "theme" from the user and generate a creative writing piece with users theme. \

Let's start with updating the flow.dag.yaml which to loads and runs the prompty as a promptflow tool. \
*(Refer to the documentation [Using prompty in a flow](https://microsoft.github.io/promptflow/how-to-guides/develop-a-prompty/use-prompty-in-flow.html?highlight=prompty) for more details.)*

flow.dag.yaml will accept the "theme" as input and will have a single "node", a python tool, that will load and run our prompty file. \
copy & paste the below .yaml into flow.dag.yaml.

```
$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
environment:
  python_requirements_txt: requirements.txt
inputs:
  theme:
    type: string
outputs:
  output_story:
    type: string
    reference: ${run_prompty.output}
nodes:
- name: run_prompty
  type: python
  source:
    type: code
    path: prompty_runner.py
  inputs:
    theme: ${inputs.theme}
```

## Write the Python 'tool' that will load and execute the Prompty file 

```
from promptflow.core import Prompty
from promptflow.core import tool
from dotenv import load_dotenv

import os

@tool
def run_prompty_tool(theme: str) -> str:
    # Get the directory of this script
    current_dir = os.path.dirname(os.path.abspath(__file__))
    # Adjust the path to point to the flow01 directory
    prompty_file_path = os.path.join(current_dir, "creative_writing_assistant.prompty")
    print(prompty_file_path)
    
    if "OPENAI_API_KEY" not in os.environ and "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
        load_dotenv()

    # Load prompty as a flow
    prompty_flow = Prompty.load(prompty_file_path)
    
    
    # Execute the flow as a function
    result = prompty_flow(theme=theme)
    return result

if __name__ == "__main__":
    import json
    import sys

    json_input = sys.argv[1]  # Assume the input is passed as a JSON string
    args = json.loads(json_input)

    result = run_prompty(**args)
    print(result)
```
This Python script defines a tool to generate creative writing stories based on a given theme using PromptFlow. \
It imports necessary modules, including Prompty, tool from promptflow.core, and load_dotenv for environment variables. \
The run_prompty_tool function, decorated with @tool, constructs the path to the creative_writing_assistant.prompty file, checks for API keys, and loads environment variables if needed. \
It then loads the Prompty flow and executes it with the provided theme, returning the result. The script also includes a main block to run the function standalone, taking JSON input from the command line and printing the result.

Save the file as **prompty_runner.py** so that it matches the .py file name previously defined in the flow.dag.yaml. \
Next let's run the flow...

In [40]:
# Run the flow locally from a Jupyter notebook
!pf flow test --flow flow01 --inputs theme="Mystery and Adventure" --verbose


[2024-05-31 09:07:47 +0300][promptflow][INFO] - pf.config.trace.destination: None
[2024-05-31 09:07:47 +0300][promptflow][INFO] - resolved tracing.trace.destination: None
Prompt flow service has started...
[2024-05-31 09:07:47 +0300][promptflow][INFO] - tracer provider is already set, will merge the resource attributes...
[2024-05-31 09:07:47 +0300][promptflow][INFO] - tracer provider is updated with resource attributes: BoundedAttributes({'service.name': 'promptflow', 'collection': 'flow01'}, maxlen=None)
[2024-05-31 09:07:47 +0300][promptflow][INFO] - have not set exporter to prompt flow service, will set it...
[2024-05-31 09:07:47 +0300][promptflow][INFO] - tracer provider is already set, will merge the resource attributes...
[2024-05-31 09:07:47 +0300][promptflow][INFO] - tracer provider is updated with resource attributes: BoundedAttributes({'service.name': 'promptflow', 'collection': 'flow01'}, maxlen=None)
[2024-05-31 09:07:47 +0300][promptflow][INFO] - exporter to prompt flow s